 # Desconsiderando os registros que possuam grande concordância ou pouquíssima concordância

In [4]:
import os
import re
import numpy as np
import math
import pandas as pd
from collections import Counter
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

############################################################################################################################
#Definição de funções
############################################################################################################################

#Função para geração do F1 médio
def geraF1(toClass, classificador): #toClass é o conjunto de treinamento
    
    f1 = 0
    
    if classificador == 'SVM':
        modelo = SVC(random_state = seed)
    elif classificador == 'DT':
        modelo = DecisionTreeClassifier(random_state = seed)
    
    #Separação do conjunto X e y
    XtoClass = toClass.iloc[:,1:]
    ytoClass = toClass.duplicata
    
    #Divisão dos conjuntos de treino e teste (20% para esse último)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(XtoClass, ytoClass, test_size=0.20, random_state=seed)
    
    #Validação cruzada
    #This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.
    kfold = StratifiedKFold(n_splits=5, random_state=seed) #n_splits igual ao tamanho do conjunto de treino?
    
    try:
        cv_results = model_selection.cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='f1')
    except:
        print("ERRO NA VALIDAÇÃO CRUZADA!")
        print("Conjunto de treinamento:")
        print(toClass)
        
    f1 = cv_results.mean()
    std = cv_results.std()
    coef_var = std/f1
#     tamTreino = len(XtoClass)
    
    return f1, std, coef_var#, tamTreino

#Função para geração do conjunto de treinamento
def geraTrainSet(ct, dir, file1):

    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)     
    
#Função para geração do conjunto de teste
def geraTestSet(ct, dir, file1):

    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)            
    
############################################################################################################################
#Repetições para os experimentos começam aqui
############################################################################################################################

#Parâmetros do usuário (Definir entrada dos dados)
estat_ord = 'min' #Estatística para ordenamento
qtd_alg = 23 #Quantidade total de algoritmos
qtd_alg_nd = 2 #Quantidade máxima de algoritmos para separar o conjunto de possíves não-duplicadas
k = 3 #Tamanho da janela
orcamento = 100 #Ou uma porcentagem da base de dados
orcamento_orig = orcamento
tam_min_ct = 20
seed = 500

etapa = '2 - AA[dg-arj]'

dirOrig = "../../csv/conjuntosDS/conjuntosDiverg/"
estat = "../../csv/estatisticaInicialDS.csv"

estatisticas = pd.read_csv(estat, index_col=['algoritmosUtilizados', 'etapa', 'permutacao'], sep=';')

arquivos = [] #Adicionado depois

for _, _, arquivo in os.walk(dirOrig):
     #print(arquivo)
     arquivos.extend(arquivo)   

for arq in arquivos:
    
    if '_NEW' in arq:
        print("##################################################################")
        print("Analisando o arquivo: {0}".format(arq))
        print("##################################################################")

        num = re.sub('diverg.*\)', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
        num = num.replace('_NEW.csv','')

        algUtl = re.sub('diverg.*\(', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
        algUtl = re.sub('\).*', r'', algUtl) #Alterar para fazer a substituição de tudo em uma linha só
        algUtl = int(algUtl)

        permutacao = int(num)
    
        linhaAtual = estatisticas.xs((algUtl, '1 - acm diverg', permutacao))    
            
        ###### Leitura do conjunto de pares conflitantes
        pc = dirOrig+arq
        
        pc = pd.read_csv(pc, sep=';', index_col=['elemento1', 'elemento2']) #pares conflitantes

        cols = list(pc.columns.values)
        cols.pop(cols.index('duplicata'))
        pc = pc[['duplicata']+cols]

        pc_aa = pc.iloc[:, :5 ] #Conjunto onde serão aplicadas as janelas deslizantes
        pc_vetores = pc.iloc[:, 5: ] #Conjunto base para compor o conjunto treinamento 

        # pc_aa = pc.loc[:, :'med' ]
        # pc_vetor = pc.loc[:, 'med': ] #Como referenciar a coluna vizinha à 'med'?

        duplicata = pc_aa.loc[:, 'duplicata' ]

        #Adicionando a coluna de duplicatas a pc_vetores
        pc_vetores = pd.concat([duplicata, pc_vetores], axis=1, ignore_index=False)

        ###### 
        
        #Separação do conjunto de pares conflitantes em dois a partir da quantidade de algoritmos 
        #que aponta o par como possível duplicata 
        #(conjunto A com quantidade de algoritmos = 1, com maioria composta por possíveis não-duplicatas,
        #e conjunto B com quantidade de algoritmos > 1, contendo mais possíveis duplicatas que o conjunto A)
        ndup = pc_aa.loc[pc['qtdAlg'] <= qtd_alg_nd]
        dup = pc_aa.loc[(pc['qtdAlg'] > qtd_alg_nd) & (pc['qtdAlg'] <= (qtd_alg - qtd_alg_nd - 1))]
        # dup = dup[pc['qtdAlg'] <= (qtd_alg - qtd_alg_nd)]

        ###### Ordenamento dos pares pela estatística selecionada
        ndup = ndup.sort_values(estat_ord) 
        dup = dup.sort_values(estat_ord, ascending=False)

        ###### Variáveis para verificar se pode deslizar a janela dentro de dup (ou ndup)

        deslz_dup = len(dup)/k
        deslz_dup

        deslz_ndup = len(ndup)/k
        deslz_ndup
        
        ###### Validação do tamanho para deslizamento
        
        #Se não houver espaço suficiente para deslizar as janelas
        if (deslz_dup < tam_min_ct/2) | (deslz_ndup < tam_min_ct/2):
            you_shall_not_pass = True
        else:
            you_shall_not_pass = False
        
        ###### Criação do dataframe que armazenará o conjunto de treinamento

        conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

        ###### Povoamento inicial sem aleatoriedade no except

        import sys

        orcamento = 100
        conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

        continua = True
        f1_anterior = 0
        f1_atual = 0
        f1_svm = 0
        f1_dt = 0

        jan_inic_ndup = 0 
        jan_fin_ndup = k

        jan_inic_dup = 0 
        jan_fin_dup = k

        cont = 0

        entrouExcept = False

        continua = True


        #Povoamento inicial com 20 pares rotulados (10 de cada)
        while (cont < tam_min_ct) & (continua):

            print("cont < 20: {0} - continua: {1}".format(cont < tam_min_ct, continua))

            print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_dup, jan_fin_ndup))

            #Desliza-se a janela
            reexecuta = True

            deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

            print("reexecuta: {0} - deslz_ndup >= 1: {1}".format(reexecuta, deslz_ndup >= 1))

            while (reexecuta) & (deslz_ndup >= 1):

                grupo = ndup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

                #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
                #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                try:

                    id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                    #Só passa pra cá se não lançar except
                    reexecuta = False
                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")
                    vetor_sel = pc_vetores.loc[id_row_sel]
                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                    orcamento -= 1


                except ValueError:

                    print("Atualizando janelas no except")
                    jan_inic_ndup = jan_inic_ndup + k
                    jan_fin_ndup = jan_fin_ndup + k

                    deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                    print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
                    print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))


                    reexecuta = True

                except:
                    print('Eita!')

            print("Possível não-duplicata")
            print(vetor_sel)

            reexecuta = True

            #SELEÇÃO DA POSSÍVEL DUPLICATA
            #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

            deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

            while (reexecuta) & (deslz_dup >= 1):

                grupo = dup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

                try:

                    id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                    #Só passa pra cá se não lançar except
                    reexecuta = False #Se selecionou um registro no comando anterior, não precisa reexecutar o while
                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
                    vetor_sel = pc_vetores.loc[id_row_sel]
                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                    orcamento -= 1

                except ValueError:

                    print("Atualizando janelas no except")
                    jan_inic_dup = jan_inic_dup + k
                    jan_fin_dup = jan_fin_dup + k

                    deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                    print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
                    print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))


                    reexecuta = True

                except:
                    print('Eita!')

            print("Possível duplicata")
            print(vetor_sel)

            #Atualização das janelas
            jan_inic_ndup = jan_inic_ndup + k
            jan_fin_ndup = jan_fin_ndup + k

            jan_inic_dup = jan_inic_dup + k
            jan_fin_dup = jan_fin_dup + k

            if(deslz_dup < 1) | (deslz_ndup < 1): #Se não houver mais espaço para deslizar as janelas
                continua = False


            cont += 1


        #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
        f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
        f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

        f1_atual = (f1_svm + f1_dt)/2
        std_atual = (std_svm + std_dt)/2
        cv_atual = (cv_svm + cv_dt)/2

        ###### Aplicação das janelas deslizantes após o povoamento incial sem aleatoriedade no except

        import warnings
        warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

        # continua = True
        max_local = 0
        igual = 0

        print("Início")
        print("orcamento: {0}".format(orcamento))
        print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

        while continua:

            f1_anterior = f1_atual

            reexecuta = True

            deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

            while (reexecuta) & (deslz_ndup >= 1):
                grupo = ndup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

                #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
                #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                try:

                    id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                    #Só passa pra cá se não lançar except
                    reexecuta = False #Se selecionou um registro no comando anterior, não precisa reexecutar o while
                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")
                    vetor_sel = pc_vetores.loc[id_row_sel]
                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                    orcamento -= 1

                except ValueError:

                    print("Atualizando janelas no except")
                    jan_inic_ndup = jan_inic_ndup + k
                    jan_fin_ndup = jan_fin_ndup + k

                    deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                    reexecuta = True

                except:
                    print('Eita!')

            print("Possível não-duplicata")
            print(vetor_sel)

            reexecuta = True

            #SELEÇÃO DA POSSÍVEL DUPLICATA
            #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

            deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

            while (reexecuta) & (deslz_dup >= 1):

                grupo = dup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

                try:

                    id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                    #Só passa pra cá se não lançar except
                    reexecuta = False
                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
                    vetor_sel = pc_vetores.loc[id_row_sel]
                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                    orcamento -= 1

                except ValueError:

                    print("Atualizando janelas no except")
                    jan_inic_dup = jan_inic_dup + k
                    jan_fin_dup = jan_fin_dup + k

                    deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                    reexecuta = True

                except:
                    print('Eita!')

            print("Possível duplicata")
            print(vetor_sel)

            #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
            f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
            f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

            f1_atual = (f1_svm + f1_dt)/2
            std_atual = (std_svm + std_dt)/2
            cv_atual = (cv_svm + cv_dt)/2 #Coeficiente de variação

            cv_atual = cv_atual*100


            if orcamento > 0 : #Se ainda tem orçamento pra gastar
                if cv_atual < 10:
                    if (f1_atual >= f1_anterior) and (igual <= 3): #Selecionam-se novos pares para rotulação
          
                        if f1_atual == f1_anterior:
                            igual += 1
                            print("igual: {0}".format(igual))
                        elif (f1_atual > f1_anterior) and (igual > 0):
                            igual = 0

                        continua = True
                    elif (f1_atual < f1_anterior):
                        max_local += 1
                        #Se entrar aqui tem que remover os últimos pares que entraram no conjunto de treinamento

                        #IMPORTANTE! Fazer com que esses devem sejam extraídos para o conjunto PM, não apenas descartados
                        conj_treino.drop(conj_treino.tail(2).index,inplace=True)

                        print("f1_atual < f1_anterior")
                        print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))

                        if (max_local <= 1):
                            f1_atual = f1_anterior
                        else:
                            f1_atual = f1_anterior
                            print('O f1 passou a piorar')
                            print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                            continua = False
                            break
                    elif (igual > 3): #Se a medida de qualidade for igual à anterior (por três vezes)
                            print('F1 convergiu!') 
                            continua = False
                            break

            else:
                continua = False
                break

            #Atualização das janelas
            jan_inic_ndup = jan_inic_ndup + k
            jan_fin_ndup = jan_fin_ndup + k

            jan_inic_dup = jan_inic_dup + k
            jan_fin_dup = jan_fin_dup + k

            if(deslz_dup < 1) | (deslz_ndup < 1): #Se não houver mais espaço para deslizar as janelas
                continua = False

            print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
            print("igual: {0}".format(igual))

        ##### Salvando os arquivos de treino e teste

        dirDest = "../../csv/conjuntosDS/treinoTeste/"
        
        abordagem = 'DS'
        
        iteracao = 1
        inspecoesManuais = orcamento_orig - orcamento
        
        duplicatas = [i for i in conj_treino.duplicata if i == True]
        duplicatas = duplicatas.count(True)
        
        nao_duplicatas = [i for i in conj_treino.duplicata if i == False]
        nao_duplicatas = nao_duplicatas.count(True)
        
        da = linhaAtual['da'].item()
        dm = duplicatas
        ndm = nao_duplicatas

        tp = float(linhaAtual['tp'].item() + dm)
        fp = float(linhaAtual['fp'].item())
        tn = float(linhaAtual['tn'].item())# + ndm) #Retirado
        fn = float(linhaAtual['fn'].item() - dm) #Adicionado
        
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        fmeasure = 2*((precision*recall)/(precision+recall))
        
        #Adicionando valor à última linha
        estatisticas.loc[(algUtl, etapa, permutacao), ['abordagem', 'iteracao', 'inspecoesManuais',
           'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
           'fp', 'tn', 'fn'] ] = ([abordagem, iteracao, inspecoesManuais,
           precision, recall, fmeasure, da, dm, ndm, tp, fp, tn, fn])
        
        dirDest = "../../csv/conjuntosDS/treinoTeste/"
#         dirDest = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/conjuntosDS/treinoTeste/"
#         dirDest = "./arqResult/csv/conjuntosDS/conjuntosDiverg/treinoTeste/"
        
        #algUtl = str(algUtl).replace('.0','')
        algUtl = str(algUtl)
        
        geraTrainSet(conj_treino, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')

        indicesCT = conj_treino.index.values.tolist()

        geral = pd.concat([pc_vetores,conj_treino]) #Concatenando pc_vetores e conj_treino

        #Resta para compor o conjunto teste tudo aquilo que está em pc_vetores, mas não em conj_treino
        conj_teste = geral.drop(indicesCT, axis='rows') 

        geraTestSet(conj_teste, dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')
        
        
        

############################################################################################################################
#Estatísticas
############################################################################################################################

#Para voltar o dataframe ao normal
estatisticas = estatisticas.reset_index(level=['algoritmosUtilizados', 'etapa', 'permutacao'])

estatisticas = estatisticas[['abordagem', 'etapa', 'algoritmosUtilizados', 'permutacao', 'iteracao', 'inspecoesManuais', 'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']]

estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']] = \
estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']].astype(int)

dirEst = "../../csv/"

# Diretório para Windows
# dirEst = "C:\Users\Diego\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\\"
# dirEst = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/"


# Diretório para Linux
# dirEst = "./arqResult/csv/"

estatisticas.to_csv(dirEst+'estatisticaInicialDS2-DgArj.csv', sep=';', index=False)

<input>:103: DeprecationWarning: invalid escape sequence \)
<input>:106: DeprecationWarning: invalid escape sequence \(
<input>:107: DeprecationWarning: invalid escape sequence \)
<input>:103: DeprecationWarning: invalid escape sequence \)
<input>:106: DeprecationWarning: invalid escape sequence \(
<input>:107: DeprecationWarning: invalid escape sequence \)
<input>:103: DeprecationWarning: invalid escape sequence \)
<input>:106: DeprecationWarning: invalid escape sequence \(
<input>:107: DeprecationWarning: invalid escape sequence \)
<ipython-input-4-cbc97c8130ec>:103: DeprecationWarning: invalid escape sequence \)
  num = re.sub('diverg.*\)', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
<ipython-input-4-cbc97c8130ec>:106: DeprecationWarning: invalid escape sequence \(
  algUtl = re.sub('diverg.*\(', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
<ipython-input-4-cbc97c8130ec>:107: DeprecationWarning: invalid escape sequence \)
  algUtl = r

##################################################################
Analisando o arquivo: diverg(10)1000_NEW.csv
##################################################################
cont < 20: True - continua: True
orcamento: 100 - jan_inic_dup: 0 - jan_fin_ndup: 3
reexecuta: True - deslz_ndup >= 1: True
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.666667
track02      0.500000
track03      0.600000
track10      0.500000
track11      0.615385
Name: (100558, 105098), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (1858, 9637), dtype: float64
cont < 20: True - continua: True
orcamento: 98 - jan_inic_dup: 3 - jan_fin_ndup: 6
reexecuta: True - deslz_ndup >= 1: True
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.304348
track02      0.571429
track03      

F1 anterior: 0.926984126984127 - F1 atual: 0.9403174603174603
igual: 0
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.545455
track02      0.785714
track03      0.500000
track10      0.444444
track11      0.692308
Name: (100558, 105076), dtype: float64
Possível duplicata
duplicata    1.000000
title        1.000000
artist       1.000000
track01      1.000000
track02      0.372093
track03      0.833333
track10      1.000000
track11      0.838710
Name: (7542, 8823), dtype: float64
f1_atual < f1_anterior
F1 anterior: 0.9403174603174603 - F1 atual: 0.9301587301587302
F1 anterior: 0.9403174603174603 - F1 atual: 0.9403174603174603
igual: 0
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.727273
track02      0.571429
track03      0.750000
track10      0.166667
track11      0.812500
Name: (100856, 103398), dtype: float64
Possível duplicata
duplicata    0.000000
title        0.495000
ar

Início
orcamento: 60
Quantidade de itens no conjunto de treinamento: 40
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.233333
track02      0.483871
track03      0.578947
track10      0.454545
track11      0.714286
Name: (100717, 106511), dtype: float64
Possível duplicata
duplicata    1.000000
title        0.966667
artist       1.000000
track01      0.800000
track02      0.928571
track03      0.916667
track10      0.714286
track11      0.866667
Name: (6944, 7731), dtype: float64
F1 anterior: 0.9457142857142857 - F1 atual: 0.9488888888888889
igual: 0
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.346154
track02      0.521739
track03      0.764706
track10      0.769231
track11      0.454545
Name: (104117, 104872), dtype: float64
Possível duplicata
duplicata    0.000000
title        0.845238
artist       1.000000
track01      0.849057
track02      0.851064
track03      0.683333

C:\Users\Diego\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:208: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


...

C:\Users\Diego\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:208: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)



duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.777778
track02      0.900000
track03      0.400000
track10      0.350000
track11      0.562500
Name: (101021, 3710), dtype: float64
Possível duplicata
duplicata    1.000000
title        1.000000
artist       1.000000
track01      0.777778
track02      0.906250
track03      0.769231
track10      0.884615
track11      0.863636
Name: (1884, 9542), dtype: float64
cont < 20: True - continua: True
orcamento: 90 - jan_inic_dup: 15 - jan_fin_ndup: 18
reexecuta: True - deslz_ndup >= 1: True
Possível não-duplicata
duplicata    0.000000
title        0.666667
artist       0.000000
track01      0.300000
track02      0.466667
track03      0.529412
track10      0.722222
track11      0.684211
Name: (100521, 100558), dtype: float64
Possível duplicata
duplicata    1.000000
title        0.666667
artist       1.000000
track01      0.933333
track02      1.000000
track03      0.947368
track10      1.000000
track11      0.94736

cont < 20: True - continua: True
orcamento: 100 - jan_inic_dup: 0 - jan_fin_ndup: 3
reexecuta: True - deslz_ndup >= 1: False
Possível não-duplicata
duplicata    1.000000
title        0.436111
artist       1.000000
track01      1.000000
track02      1.000000
track03      1.000000
track10      0.920000
track11      1.000000
Name: (10312, 9807), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (2187, 2418), dtype: float64
ERRO NA VALIDAÇÃO CRUZADA!
Conjunto de treinamento:
              duplicata  title  artist  track01  track02  track03  track10  \
(2187, 2418)        1.0    1.0     0.0      1.0      1.0      1.0      1.0   

              track11  
(2187, 2418)      1.0  


C:\Users\Diego\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:208: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


UnboundLocalError: local variable 'cv_results' referenced before assignment